##### Feature selection method 
### Forrward selection 
### Backward elemination

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Cars93.csv")
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,NaN,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  93 non-null     int64  
 1   Manufacturer        93 non-null     object 
 2   Model               93 non-null     object 
 3   Type                93 non-null     object 
 4   Min.Price           93 non-null     float64
 5   Price               93 non-null     float64
 6   Max.Price           93 non-null     float64
 7   MPG.city            93 non-null     int64  
 8   MPG.highway         93 non-null     int64  
 9   AirBags             56 non-null     object 
 10  DriveTrain          93 non-null     object 
 11  Cylinders           93 non-null     object 
 12  EngineSize          93 non-null     float64
 13  Horsepower          93 non-null     int64  
 14  RPM                 93 non-null     int64  
 15  Rev.per.mile        93 non-null     int64  
 16  Man.trans.

##### check for missing values 

In [6]:
s=df.isna().sum()
s[s>0]

AirBags           37
Rear.seat.room     2
Luggage.room      11
dtype: int64

#### Seperate X and Y

In [8]:
X = df.drop(columns=["id","Weight"])
Y= df[["Weight"]]

In [9]:
X.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,NaN,Front,...,13.2,5,177,102,68,37,26.5,11.0,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,non-USA,BMW 535i


In [10]:
Y.head

<bound method NDFrame.head of     Weight
0     2705
1     3560
2     3375
3     3405
4     3640
..     ...
88    3960
89    2985
90    2810
91    2985
92    3245

[93 rows x 1 columns]>

#### Seperate catagorical and continues features

In [12]:
cat = list(X.columns[X.dtypes=="object"])
con = list(X.columns[X.dtypes!="object"])

In [13]:
cat

['Manufacturer',
 'Model',
 'Type',
 'AirBags',
 'DriveTrain',
 'Cylinders',
 'Man.trans.avail',
 'Origin',
 'Make']

In [15]:
con

['Min.Price',
 'Price',
 'Max.Price',
 'MPG.city',
 'MPG.highway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turn.circle',
 'Rear.seat.room',
 'Luggage.room']

#### Featre selection pipeline

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

In [24]:
# Numerical pipeline 1 
num_pipe1 = Pipeline(steps=[("imputer",SimpleImputer(strategy="median")),
                            ("scaler",StandardScaler())])
# catagorical pipeline
cat_pipe1 = Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),
                            ("ordinal",OrdinalEncoder())])
# combine both the pipeline
pre1 = ColumnTransformer([("num",num_pipe1,con),
                          ("cat",cat_pipe1,cat)])
pre1


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Min.Price', 'Price', 'Max.Price', 'MPG.city',
                                  'MPG.highway', 'EngineSize', 'Horsepower',
                                  'RPM', 'Rev.per.mile', 'Fuel.tank.capacity',
                                  'Passengers', 'Length', 'Wheelbase', 'Width',
                                  'Turn.circle', 'Rear.seat.room',
                                  'Luggage.room']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Manufacturer', 'Model', 'Type', 'AirBags',
                                  'DriveTrain', 'Cylinders', 'Man.trans.avail',
                                  'Origin', 'Make'])])

#### fit transform your data

In [69]:
X_pre =pre1.fit_transform(X)

In [70]:
X_pre

array([[-0.48578741, -0.37572014, -0.28246529, ...,  1.        ,
         1.        ,  0.        ],
       [ 1.38801699,  1.49784409,  1.53140881, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.00865782,  0.99822696,  0.94805231, ...,  1.        ,
         1.        ,  3.        ],
       ...,
       [ 0.66378585,  0.39452293,  0.16416702, ...,  1.        ,
         1.        , 87.        ],
       [ 0.53733279,  0.33207079,  0.14593713, ...,  1.        ,
         1.        , 91.        ],
       [ 0.88220476,  0.7484184 ,  0.60168439, ...,  1.        ,
         1.        , 92.        ]])

In [32]:
cols = pre1.get_feature_names_out()

In [71]:
cols

array(['num__Min.Price', 'num__Price', 'num__Max.Price', 'num__MPG.city',
       'num__MPG.highway', 'num__EngineSize', 'num__Horsepower',
       'num__RPM', 'num__Rev.per.mile', 'num__Fuel.tank.capacity',
       'num__Passengers', 'num__Length', 'num__Wheelbase', 'num__Width',
       'num__Turn.circle', 'num__Rear.seat.room', 'num__Luggage.room',
       'cat__Manufacturer', 'cat__Model', 'cat__Type', 'cat__AirBags',
       'cat__DriveTrain', 'cat__Cylinders', 'cat__Man.trans.avail',
       'cat__Origin', 'cat__Make'], dtype=object)

In [72]:
X_pre = pd.DataFrame(X_pre,columns=cols)
X_pre.head()

,num__Min.Price,num__Price,num__Max.Price,num__MPG.city,num__MPG.highway,num__EngineSize,num__Horsepower,num__RPM,num__Rev.per.mile,num__Fuel.tank.capacity,...,num__Luggage.room,cat__Manufacturer,cat__Model,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin,cat__Make
0,-0.485787,-0.375720,-0.282465,0.471312,0.360925,-0.841022,-0.073484,1.717489,1.129530,-1.062184,...,-1.037572,0.0,48.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0
1,1.388017,1.497844,1.531409,-0.781032,-0.770514,0.515869,1.078322,0.369586,0.005661,0.409445,...,0.391972,0.0,55.0,2.0,0.0,1.0,3.0,1.0,1.0,1.0
2,1.008658,0.998227,0.948052,-0.423219,-0.581941,0.128186,0.540813,0.369586,-0.105713,0.072197,...,0.034586,1.0,8.0,0.0,1.0,1.0,3.0,1.0,1.0,3.0
3,1.571949,1.893374,2.069191,-0.602126,-0.581941,0.128186,0.540813,0.369586,0.410659,1.359872,...,1.106743,1.0,0.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0
4,0.755752,1.091905,1.303535,-0.065407,0.172352,0.806631,1.231897,0.706562,0.430909,1.359872,...,-0.322800,2.0,5.0,2.0,1.0,2.0,1.0,1.0,1.0,4.0


### Feature selection process

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector 

In [74]:
lr = LinearRegression()
sel = SequentialFeatureSelector(lr,direction="forward")
sel_features = sel.fit_transform(X_pre,Y)
sel_cols = sel.get_feature_names_out()
sel_cols

array(['num__MPG.highway', 'num__Horsepower', 'num__RPM',
       'num__Fuel.tank.capacity', 'num__Passengers', 'num__Length',
       'num__Wheelbase', 'num__Width', 'num__Rear.seat.room',
       'cat__Manufacturer', 'cat__Type', 'cat__Cylinders', 'cat__Origin'],
      dtype=object)

In [75]:
len(sel_cols)

13

In [76]:
sel_cols[0].split("__")[1]

'MPG.highway'

In [77]:
imp_cols = []
for i in sel_cols:
    s=i.split("__")[1]
    imp_cols.append(s)
    

In [56]:
imp_cols

['MPG.highway',
 'Horsepower',
 'RPM',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Rear.seat.room',
 'Manufacturer',
 'Type',
 'Cylinders',
 'Origin']

In [78]:
X_Sel = X[imp_cols]
X_Sel

,MPG.highway,Horsepower,RPM,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Rear.seat.room,Manufacturer,Type,Cylinders,Origin
0,31,140,6300,13.2,5,177,102,68,26.5,Acura,Small,4,non-USA
1,25,200,5500,18.0,5,195,115,71,30.0,Acura,Midsize,6,non-USA
2,26,172,5500,16.9,5,180,102,67,28.0,Audi,Compact,6,non-USA
3,26,172,5500,21.1,6,193,106,70,31.0,Audi,Midsize,6,non-USA
4,30,208,5700,21.1,4,186,109,69,27.0,BMW,Midsize,4,non-USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,21,109,4500,21.1,7,187,115,72,34.0,Volkswagen,Van,5,non-USA
89,30,134,5800,18.5,5,180,103,67,31.5,Volkswagen,Compact,4,non-USA
90,25,178,5800,18.5,4,159,97,66,26.0,Volkswagen,Sporty,6,non-USA
91,28,114,5400,15.8,5,190,104,67,29.5,Volvo,Compact,4,non-USA


 ### create final pipeline
~~~
seperate cat_selected and con_selected
~~~

In [79]:
cat_sel = list(X_Sel.columns[X_Sel.dtypes=="object"])
con_sel = list(X_Sel.columns[X_Sel.dtypes!="object"])

In [80]:
cat_sel

['Manufacturer', 'Type', 'Cylinders', 'Origin']

In [61]:
con_sel

['MPG.highway',
 'Horsepower',
 'RPM',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Rear.seat.room']

In [81]:
from sklearn.preprocessing import OneHotEncoder

In [83]:
# Num pipeline
num_pipe2 = Pipeline(steps=[("imputer",SimpleImputer(strategy="median")),
                            ("scaler",StandardScaler())])
# Cat pipeline
cat_pipe2 = Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),
                            ("ohe",OneHotEncoder(handle_unknown="ignore"))])
# combine both pipeline

pre2 = ColumnTransformer([("num",num_pipe2,con_sel),
                          ("cat",cat_pipe2,cat_sel)])
pre2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['MPG.highway', 'Horsepower', 'RPM',
                                  'Fuel.tank.capacity', 'Passengers', 'Length',
                                  'Wheelbase', 'Width', 'Rear.seat.room']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Manufacturer', 'Type', 'Cylinders',
                                  'Origin'])])

In [86]:
X_sel_pre = pre2.fit_transform(X_Sel).toarray()
X_sel_pre

array([[ 0.36092485, -0.07348445,  1.71748854, ...,  0.        ,
         0.        ,  1.        ],
       [-0.77051373,  1.07832243,  0.36958614, ...,  0.        ,
         0.        ,  1.        ],
       [-0.58194063,  0.54081255,  0.36958614, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.77051373,  0.65599324,  0.87504954, ...,  0.        ,
         0.        ,  1.        ],
       [-0.20479444, -0.57260077,  0.20109834, ...,  0.        ,
         0.        ,  1.        ],
       [-0.20479444,  0.46402543,  1.54900074, ...,  0.        ,
         0.        ,  1.        ]])

In [88]:
final_cols = pre2.get_feature_names_out()
final_cols

array(['num__MPG.highway', 'num__Horsepower', 'num__RPM',
       'num__Fuel.tank.capacity', 'num__Passengers', 'num__Length',
       'num__Wheelbase', 'num__Width', 'num__Rear.seat.room',
       'cat__Manufacturer_Acura', 'cat__Manufacturer_Audi',
       'cat__Manufacturer_BMW', 'cat__Manufacturer_Buick',
       'cat__Manufacturer_Cadillac', 'cat__Manufacturer_Chevrolet',
       'cat__Manufacturer_Chrylser', 'cat__Manufacturer_Chrysler',
       'cat__Manufacturer_Dodge', 'cat__Manufacturer_Eagle',
       'cat__Manufacturer_Ford', 'cat__Manufacturer_Geo',
       'cat__Manufacturer_Honda', 'cat__Manufacturer_Hyundai',
       'cat__Manufacturer_Infiniti', 'cat__Manufacturer_Lexus',
       'cat__Manufacturer_Lincoln', 'cat__Manufacturer_Mazda',
       'cat__Manufacturer_Mercedes-Benz', 'cat__Manufacturer_Mercury',
       'cat__Manufacturer_Mitsubishi', 'cat__Manufacturer_Nissan',
       'cat__Manufacturer_Oldsmobile', 'cat__Manufacturer_Plymouth',
       'cat__Manufacturer_Pontiac', 'cat__

In [91]:
X_sel_pre = pd.DataFrame(X_sel_pre,columns=final_cols)
X_sel_pre.head()

,num__MPG.highway,num__Horsepower,num__RPM,num__Fuel.tank.capacity,num__Passengers,num__Length,num__Wheelbase,num__Width,num__Rear.seat.room,cat__Manufacturer_Acura,...,cat__Type_Sporty,cat__Type_Van,cat__Cylinders_3,cat__Cylinders_4,cat__Cylinders_5,cat__Cylinders_6,cat__Cylinders_8,cat__Cylinders_rotary,cat__Origin_USA,cat__Origin_non-USA
0,0.360925,-0.073484,1.717489,-1.062184,-0.083243,-0.427186,-0.286932,-0.366184,-0.449727,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.770514,1.078322,0.369586,0.409445,-0.083243,0.812171,1.629649,0.431983,0.740403,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-0.581941,0.540813,0.369586,0.072197,-0.083243,-0.220626,-0.286932,-0.632239,0.060329,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.581941,0.540813,0.369586,1.359872,0.884457,0.674465,0.302785,0.165927,1.080441,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.172352,1.231897,0.706562,1.359872,-1.050944,0.192493,0.745073,-0.100128,-0.279708,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


### Train test split

In [93]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X_sel_pre,Y,test_size=0.2,random_state=21)

### Create model

In [94]:
from sklearn.linear_model import LinearRegression

In [101]:
model = LinearRegression()
model.fit(xtrain,ytrain)

LinearRegression()

#### Check results in training and testing

In [97]:
model.score(xtrain,ytrain)

0.9867534860492617

In [100]:
model.score(xtest,ytest)

0.9503837893592195

In [103]:
import os
os.add_dll_directory("/work/Arvind/PythonStudy/vscode/repository/Automated_Preprocessing_Using_Pipeline/")
#evaluate_model(xtrain,xtest,ytrain,ytest,model)

<AddedDllDirectory('/work/Arvind/PythonStudy/vscode/repository/Automated_Preprocessing_Using_Pipeline/')>